In [ ]:
import pypsa
from pypsa.statistics import get_transmission_branches, get_transmission_carriers
from cartopy import crs as ccrs
import matplotlib.pyplot as plt
import geopandas as gpd


regions = gpd.read_file(
    "/home/fabian/papers/co2-network/resources/regions_onshore_elec_s_40.geojson"
)
regions = regions.set_index("name")

n = pypsa.Network("/home/fabian/papers/co2-network/results/full/networks/40_nodes.nc")
# TODO: this should all go into pypsa-eur network building
n.add("Bus", "EU", x=-10, y=48)
n.buses.location.fillna("EU", inplace=True)
n.buses.loc["co2 atmosphere", "location"] = "EU"
n.buses["x"] = n.buses.location.map(n.buses.x)
n.buses["y"] = n.buses.location.map(n.buses.y)


s = n.statistics
g = s.groupers


bus_carrier = "H2"
transmission_carriers = get_transmission_carriers(n, bus_carrier).unique(1)

supply = s.supply(bus_carrier=bus_carrier, groupby=g.get_bus_and_carrier)
supply = supply.rename(n.buses.location, level=1)
supply = supply.drop(transmission_carriers, level=2, errors="ignore")

bus_sizes = supply.groupby(level=[1, 2]).sum()
flow = s.transmission(groupby=False, bus_carrier=bus_carrier)


bus_scale = 5e-9
flow_scale = 5e-6
branch_scale = 2e-8


n.plot(
    projection=ccrs.EqualEarth(),
    bus_sizes=bus_sizes * bus_scale,
    bus_alpha=0.8,
    bus_colors=n.carriers.set_index("nice_name").color,
    flow=flow * flow_scale,
    link_widths=flow.abs().get("Link", 0) * branch_scale,
    line_widths=flow.abs().get("Line", 0) * branch_scale,
)

plt.show()

In [ ]:
df = s.optimal_capacity(
    bus_carrier=bus_carrier, groupby=g.get_bus_and_carrier_and_bus_carrier
)
bus_sizes = (
    df.rename(n.buses.location, level=1)
    .drop("Store", level=0, errors="ignore")
    .drop(transmission_carriers, level=2)
    .groupby(level=[1, 2])
    .sum()
)

transmission_branches = get_transmission_branches(n, bus_carrier=bus_carrier)
df = s.optimal_capacity(groupby=False)
branch_widths = df.loc[transmission_branches]

df = s.optimal_capacity(bus_carrier=bus_carrier, groupby="bus", storage=True)
df = df.rename(n.buses.location, level=1)
regions = regions.assign(color=df.groupby(level=1).sum())

bus_scale = 1e-5
branch_scale = 1e-4


fig, ax = plt.subplots(subplot_kw={"projection": ccrs.EqualEarth()})
n.plot(
    ax=ax,
    bus_sizes=bus_sizes * bus_scale,
    bus_alpha=0.8,
    bus_colors=n.carriers.set_index("nice_name").color,
    link_widths=branch_widths.get("Link", 0) * branch_scale,
    line_widths=branch_widths.get("Line", 0) * branch_scale,
)
regions.plot(
    ax=ax,
    column="color",
    cmap="Blues",
    vmin=0,
    facecolor="whitesmoke",
    edgecolor="black",
    aspect="equal",
    transform=ccrs.PlateCarree(),
    linewidth=0.0,
    legend=True,
)

plt.show()  